In [ ]:
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.svm import SVR
import numpy as np

In [ ]:
df = pd.read_csv("train_file.csv")

In [ ]:
columns = df.columns

In [ ]:
columns

In [ ]:
df = df.drop(["GeoLocation", "Greater_Risk_Question", "Patient_ID"], axis=1)

In [ ]:
dataset = df
#dataset = df.drop("Description", axis=1)

In [ ]:
le_col = ["LocationDesc","Sex", "Race","StratificationType","QuestionCode"]

In [ ]:
from sklearn import preprocessing
le = preprocessing.LabelEncoder()

In [ ]:
for i in le_col:
    dataset[i] = le.fit_transform(dataset[i])

In [ ]:
dataset["Description"][1]

In [ ]:
from nltk.corpus import stopwords
sw = stopwords.words("english")

import re
from nltk.stem.snowball import SnowballStemmer
import string

def clean_text(text):

    ## Remove puncuation
    text = text.translate(string.punctuation)

    ## Convert words to lower case and split them
    text = text.lower().split()

    ## Remove stop words
    text = [w for w in text if not w in sw and len(w) >= 3]

    text = " ".join(text)
    ## Clean the text
    text = re.sub(r"[^A-Za-z0-9^,!.\/'+-=]", " ", text)
    text = re.sub(r"what's", "what is ", text)
    text = re.sub(r"\'s", " ", text)
    text = re.sub(r"\'ve", " have ", text)
    text = re.sub(r"n't", " not ", text)
    text = re.sub(r"i'm", "i am ", text)
    text = re.sub(r"\'re", " are ", text)
    text = re.sub(r"\'d", " would ", text)
    text = re.sub(r"\'ll", " will ", text)
    text = re.sub(r",", " ", text)
    text = re.sub(r"\.", " ", text)
    text = re.sub(r"!", " ! ", text)
    text = re.sub(r"\/", " ", text)
    text = re.sub(r"\^", " ^ ", text)
    text = re.sub(r"\+", " + ", text)
    text = re.sub(r"\-", " - ", text)
    text = re.sub(r"\=", " = ", text)
    text = re.sub(r"'", " ", text)
    text = re.sub(r"(\d+)(k)", r"\g<1>000", text)
    text = re.sub(r":", " : ", text)
    text = re.sub(r" e g ", " eg ", text)
    text = re.sub(r" b g ", " bg ", text)
    text = re.sub(r" u s ", " american ", text)
    text = re.sub(r"\0s", "0", text)
    text = re.sub(r" 9 11 ", "911", text)
    text = re.sub(r"e - mail", "email", text)
    text = re.sub(r"j k", "jk", text)
    text = re.sub(r"\s{2,}", " ", text)
    ## Stemming
    txt = text.split()
    text = [i for i in txt if i.isalpha()]
    stemmer = SnowballStemmer('english')
    stemmed_words = [stemmer.stem(word) for word in text]
    text = " ".join(stemmed_words)

    return text

In [ ]:
dataset["Description"] = dataset["Description"].apply(clean_text)

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfv = TfidfVectorizer(min_df=3, max_features=1000,
strip_accents='unicode', analyzer='word', token_pattern=r'\w{1,}',
ngram_range=(1, 3), use_idf=1, smooth_idf=1, sublinear_tf=1)

tfv.fit(list(dataset["Description"]))
desc = tfv.transform(dataset["Description"])

In [ ]:
# truncate the matrix/ array to 30 column
from sklearn.decomposition import TruncatedSVD
svd = TruncatedSVD(n_components=30)
svd.fit(desc)
desc = svd.transform(desc)

desc = pd.DataFrame(desc, columns=['nlp_{}'.format(i) for i in range(30)])
dataset = dataset.drop("Description", axis=1)

In [ ]:
final_df = pd.concat([dataset, desc],axis=1, sort=False)

In [ ]:
final_df.head()

In [ ]:
final_df["YEAR"] = final_df["YEAR"]-1991

In [ ]:
final_df["Sample_Size"] = final_df["Sample_Size"] - final_df["Sample_Size"].min()

In [ ]:
Y = final_df["Greater_Risk_Probability"]
final_df = final_df.drop("Greater_Risk_Probability", axis=1)

In [ ]:
final_df.head()

In [ ]:
X = dataset

In [ ]:
Y = Y/10

In [ ]:
Y = Y.round(0) 

In [ ]:
Y = Y.astype(int)

In [ ]:
scaler = StandardScaler()
X = scaler.fit_transform(X)

In [ ]:

X_train, X_test, Y_train, Y_test = train_test_split(X, Y ,test_size=0.25, random_state=42)

In [ ]:
svr_rbf = SVR(kernel='poly', C=100, gamma=0.1, epsilon=.1)
svr_rbf.fit(X_train, Y_train)
Y_svr = svr_rbf.predict(X_test)

In [ ]:
print("Mean squared error: %.2f" % np.mean((Y_test - Y_svr) ** 2))

In [ ]:
Y_test = Y_test.tolist()

In [ ]:
test_data = pd.read_csv("test_file.csv")

In [ ]:
test_data = test_data.drop(["GeoLocation", "Greater_Risk_Question", "Patient_ID"], axis=1)

In [ ]:
for i in le_col:
    test_data[i] = le.fit_transform(test_data[i])

In [ ]:
test_sol = svr_rbf.predict(test_data)

In [ ]:
df_sub = pd.read_csv("sample_submission.csv")

In [ ]:
df_sub.head()

In [ ]:
Pat_ID = pd.read_csv("test_file.csv")

In [ ]:
Pat_ID = Pat_ID["Patient_ID"]

In [ ]:
type(Pat_ID)

In [ ]:
Pat_ID = Pat_ID.tolist()

In [ ]:
test_sol = test_sol.tolist()

In [ ]:
for i in range(len(test_sol)):
    test_sol[i] = test_sol[i]/100000000

In [ ]:
for i in range(len(test_sol)):
    if test_sol[i] <0:
        test_sol[i] = 0

In [ ]:
df_result = pd.DataFrame({"Patient_ID":Pat_ID,"Greater_Risk_Probability":test_sol}, index=None)

In [ ]:
df_result.describe()

In [ ]:
df_final_1 = pd.merge(pd.DataFrame(df_result["Patient_ID"]),pd.DataFrame(df_result["Greater_Risk_Probability"]),left_index=True, right_index=True)

In [ ]:
df_final_1.to_csv("submission_3.csv", index=False)